In [101]:
from langchain.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI
import os

# OpenAI API configuration
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [102]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key= os.environ["OPENAI_API_KEY"]
)

In [103]:
neo4j_url = "bolt://localhost:7687"
neo4j_user = "neo4j"
neo4j_password = "12345678"
graph = Neo4jGraph(url=neo4j_url, username=neo4j_user, password=neo4j_password)

In [104]:
# Cypher generation prompt
cypher_generation_template = """
You are an expert Neo4j Cypher translator who converts English to Cypher based on the Neo4j Schema provided, following the instructions below:
1. Generate Cypher query compatible ONLY for Neo4j Version 5
2. Do not use EXISTS, SIZE, HAVING keywords in the cypher. Use alias when using the WITH keyword
3. Use only nodes and relationships mentioned in the schema
4. Always do a case-insensitive and fuzzy search for any properties related search. Eg: to search for a Person, use `toLower(entity.name) contains 'neo4j'`. 
5. Never use relationships that are not mentioned in the given schema
6. When asked about entities, Match the properties using case-insensitive matching, E.g, to find a person named Radagon , use `toLower(entity.name) contains 'radagon'`.
7. When asked about a person, Match the label property with the word "person", E.g, to find a person named Marika , use `toLower(entity.label) = 'person'`.
7. When asked about a place, Match the label property with the word "place", E.g, to find a place named limgrave , use `toLower(entity.label) = 'place'`.
8. If a person, place, object or event does not match an entity, Try matching the description property or the metadata property of a relationship using case-insensitive matching, E.g, to find information about Blackguard Big Boggart, use toLower(r.description) contains 'blackguard big boggart' OR toLower(r.metadata) contains 'blackguard big boggart'.
9. When asked about any information of an entity, Do not simply give the entity label. Try to get the answer from the entity's relationship description or metadata property

schema: {schema}

Examples:
Question: Who is Blackguard Big Boggart?
MATCH (e:Entity)-[r:RELATED]->(re:Entity)
WHERE toLower(r.description) CONTAINS 'blackguard big boggart'
OR toLower(r.metadata) CONTAINS 'blackguard big boggart'
RETURN e.name, r.metadata, r.description, re.name

Question: Where is Limgrave?
MATCH (e:Entity)-[r:RELATED]->(re:Entity)
WHERE toLower(e.label) = 'place' AND toLower(e.name) = "limgrave"
RETURN e.name, r.metadata, r.description, re.name

Question: List all the locations in elden ring
Answer: ```MATCH (e:Entity)
WHERE e.label ="Place"
RETURN e```

Question: {question}
"""

cypher_prompt = PromptTemplate(
    template = cypher_generation_template,
    input_variables = ["schema", "question"]
)

CYPHER_QA_TEMPLATE = """You are an assistant that helps to form nice and human understandable answers.
The information part contains the provided information that you must use to construct an answer.
The provided information is authoritative, you must never doubt it or try to use your internal knowledge to correct it.
Make the answer sound as a response to the question. Do not mention that you based the result on the given information.
If the provided information is empty, say that you don't know the answer.
Final answer should be easily readable and structured.
Information:
{context}

Question: {question}
Helpful Answer:"""

qa_prompt = PromptTemplate(
    input_variables=["context", "question"], template=CYPHER_QA_TEMPLATE
)

In [105]:
def query_graph(user_input):
    chain = GraphCypherQAChain.from_llm(
        llm=llm,
        graph=graph,
        verbose=True,
        return_intermediate_steps=True,
        cypher_prompt=cypher_prompt,
        qa_prompt=qa_prompt
        )
    result = chain(user_input)
    return result

In [106]:
def refine_query(previous_query, user_input):

   cypher_refine_template = f"""" 
   Context: I am working with a Neo4j database containing information.

   Initial Cypher Query:
      {previous_query}
   """

   cypher_refine_template += """
   Problem:
   The above Cypher Query returned no results. 
   I need to refine this query to achieve to answer the question {question}: 

   Schema: {schema}

   Request:
   Can you please refine the Initial Cypher Query to answer the question?
   """
    
   cypher_refine_prompt = PromptTemplate(
      input_variables=["schema", "question"], template=cypher_refine_template
   )

   graph = Neo4jGraph(url=neo4j_url, username=neo4j_user, password=neo4j_password)
   chain = GraphCypherQAChain.from_llm(
       llm=llm,
       graph=graph,
       verbose=True,
       return_intermediate_steps=True,
       cypher_prompt=cypher_refine_prompt
   )
   
   print(cypher_refine_prompt.format(question=user_input, schema=graph.schema, prevquery=previous_query))
   result = chain(user_input)
   return result

In [110]:
user_input = "What is an Amber Egg?"

In [111]:
result = query_graph(user_input)
intermediate_steps = result["intermediate_steps"]
cypher_query = intermediate_steps[0]["query"]
database_results = intermediate_steps[1]["context"]
answer = result["result"]   



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (e:Entity)-[r:RELATED]->(re:Entity)
WHERE toLower(re.name) CONTAINS 'amber egg'
OR toLower(r.description) CONTAINS 'amber egg'
OR toLower(r.metadata) CONTAINS 'amber egg'
RETURN e.name, r.metadata, r.description, re.name

Full Context:
[{'e.name': 'Tarnished', 'r.metadata': '{"summary": "The Tarnished discovers Rennala holding the Amber Egg.", "generated_at": "2024-08-24 20:26:22.238546"}', 'r.description': 'The Tarnished is in a situation where they find Rennala.', 're.name': 'Rennala'}, {'e.name': 'Radagon', 'r.metadata': '{"summary": "Radagon gave Rennala an Amber Egg.", "generated_at": "2024-08-24 20:26:22.238546"}', 'r.description': 'Radagon and Rennala are involved in an event where Radagon gifts an Amber Egg to Rennala.', 're.name': 'Rennala'}, {'e.name': 'Rennala', 'r.metadata': '{"summary": "The Tarnished discovers Rennala holding the Amber Egg.", "generated_at": "2024-08-24 20:26:22.238546"}', 'r.desc

In [112]:
answer

'The Amber Egg is a significant object in the context of the characters Radagon and Rennala. Radagon gifted the Amber Egg to Rennala, and at one point, the Tarnished discovers Rennala cradling it. This egg symbolizes a connection between these characters, with Radagon being the giver and Rennala the recipient.'

In [82]:
result = ""
if answer == "I don't know the answer.":
    print("No answer!")
    result = refine_query(cypher_query[6:], user_input)
else:
    print(answer)

No answer!
" 
   Context: I am working with a Neo4j database containing information.

   Initial Cypher Query:
      
MATCH (e:Entity)-[r:RELATED]->(re:Entity)
WHERE toLower(r.description) CONTAINS 'apple'
OR toLower(r.metadata) CONTAINS 'apple'
RETURN e.name, r.metadata, r.description, re.name

   
   Problem:
   The above Cypher Query returned no results. 
   I need to refine this query to achieve to answer the question What is an Apple?: 

   Schema: Node properties:
Entity {name: STRING, label: STRING}
Relationship properties:
RELATED {description: STRING, metadata: STRING, order: INTEGER}
The relationships:
(:Entity)-[:RELATED]->(:Entity)

   Request:
   Can you please refine the Initial Cypher Query to answer the question?
   


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (e:Entity)-[r:RELATED]->(re:Entity)
WHERE toLower(e.name) = 'apple' 
   OR toLower(r.description) CONTAINS 'apple' 
   OR toLower(r.metadata) CONTAINS 'apple' 
   OR toLower(re.name

In [85]:
result

{'query': 'What is an Apple?',
 'result': "I don't know the answer.",
 'intermediate_steps': [{'query': "cypher\nMATCH (e:Entity)-[r:RELATED]->(re:Entity)\nWHERE toLower(e.name) = 'apple' \n   OR toLower(r.description) CONTAINS 'apple' \n   OR toLower(r.metadata) CONTAINS 'apple' \n   OR toLower(re.name) = 'apple'\nRETURN e.name AS EntityName, r.metadata AS RelationshipMetadata, r.description AS RelationshipDescription, re.name AS RelatedEntityName\n"},
  {'context': []}]}

In [100]:
oldquery = cypher_query
print(f"""{oldquery}""")

cypher
MATCH (e:Entity)-[r:RELATED]->(re:Entity)
WHERE toLower(r.description) CONTAINS 'apple'
OR toLower(r.metadata) CONTAINS 'apple'
RETURN e.name, r.metadata, r.description, re.name



In [96]:
newquery = result["intermediate_steps"][0]['query']

In [97]:
print(f"""{newquery}""")

cypher
MATCH (e:Entity)-[r:RELATED]->(re:Entity)
WHERE toLower(e.name) = 'apple' 
   OR toLower(r.description) CONTAINS 'apple' 
   OR toLower(r.metadata) CONTAINS 'apple' 
   OR toLower(re.name) = 'apple'
RETURN e.name AS EntityName, r.metadata AS RelationshipMetadata, r.description AS RelationshipDescription, re.name AS RelatedEntityName

